In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

import time
from multiprocessing import Pool
from tqdm.auto import tqdm
import re
from copy import deepcopy

import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt

import noctiluca as nl
import bayesmsd

In [2]:
filename = '../data/20250411_chromatin_dynamics_all_data.h5'
data       = nl.io.load.hdf5(filename)['data']

In [3]:
n_subsample = 4 # cut off the "kink" at the beginning of MINFLUX data
def subsample(traj):
    out = nl.Trajectory(traj[::n_subsample])
    out.meta['Δt'] = n_subsample*traj.meta['Δt']
    return out

In [4]:
data.makeSelection('minflux')
data.apply(subsample, inplace=True)

# Fits

In [5]:
def chop(traj, dt=None, L=200, Fmin=2):
    if dt is None:
        dt = traj.meta['Δt']
    
    def chop_traj(traj, dt=dt):
        if 'Δt' in traj.meta:
            dt = traj.meta['Δt']
            
        chops = []
        i0 = 0
        while i0 < len(traj):
            i1 = i0+L
            chop = traj.data[:, i0:min(i1, len(traj)), :]
            try:
                t_start = np.nonzero(~np.any(np.isnan(chop), axis=(0, 2)))[0][0]
            except IndexError: # no valid entries in this chop
                new_traj = nl.Trajectory(chop[:, [0]])
            else:
                new_traj = nl.Trajectory(chop[:, t_start:])
                
            new_traj.meta['Δt'] = dt
            chops.append(new_traj)

            i0 = i1
            
        return chops
    
    chops = chop_traj(traj)
    out = nl.TaggedSet(chops, hasTags=False)
    while len(chops) > 1:
        cg_traj = nl.Trajectory(np.stack([traj.data[:, 0] for traj in chops], axis=1))
        cg_traj.meta['Δt'] = L*chops[0].meta['Δt']
        chops = chop_traj(cg_traj)
        for traj in chops:
            out.add(traj)
    
    # Clean out useless trajectories
    out.makeSelection(lambda traj, _: traj.F < Fmin)
    out.deleteSelection()
    return out

In [6]:
ct = 'U2OS'
bar = tqdm()

fits = {}
for treatment in ['ctrl', 'DRB', 'TSA', 'ICRF']:

    cond = ', '.join(['H2B', ct, treatment])
    fits[treatment] = {
        'single' : {},
        'joints' : {},
    }

    # Minflux
    data.makeSelection(['minflux', cond], logic=all)
    dt = data[0].meta['Δt']

    fitdata = nl.TaggedSet()
    for traj in data:
        fitdata |= chop(traj.rescale(1e6, keepmeta=['Δt']))

    with nl.Parallelize():
        _ = nl.analysis.MSD(fitdata, chunksize=10, show_progress=True)

    fit = bayesmsd.lib.NPFit(fitdata, motion_blur_f=dt/n_subsample, parametrization='(log(αΓ), α)')
    fit.parameters['log(σ²) (dim 1)'].fix_to = 'log(σ²) (dim 0)'
    fit.likelihood_chunksize = 200

    fits[treatment]['single'][f'minflux'] = fit

    bar.update()

    # Conventional
    for dt_tag in ['100ms', '2s']:
        data.makeSelection(['SPT', dt_tag, cond], logic=all)
        dt = data[0].meta['Δt']
        tau_e = 0.08671 # same exposure for both conditions

        fitdata = data.apply(lambda traj : traj.relative(keepmeta=['MSD', 'Δt']), inplace=False)

        fit = bayesmsd.lib.NPFit(fitdata, motion_blur_f=tau_e, parametrization='(log(αΓ), α)')
        fit.parameters['log(σ²) (dim 1)'].fix_to = 'log(σ²) (dim 0)'
        fit.likelihood_chunksize = 100

        fits[treatment]['single'][f'SPT-{dt_tag}'] = fit

        bar.update()

    # Assemble list of fit(group)s to run
    groups = {
        'minflux'       : ['minflux'],
        'SPT 100ms'     : ['SPT-100ms'],
        'SPT 2s'        : ['SPT-2s'],
        'SPT'           : ['SPT-100ms', 'SPT-2s'],
        'minflux + SPT' : ['minflux', 'SPT-100ms', 'SPT-2s'],
    }

    for groupname in groups:
        fits_dict = fits[treatment]['single']

        fit = bayesmsd.FitGroup({name : fits_dict[name] for name in groups[groupname]})
        fit.parameters['α']       = deepcopy(fits_dict['minflux'].parameters[      'α (dim 0)'])
        fit.parameters['log(αΓ)'] = deepcopy(fits_dict['minflux'].parameters['log(αΓ) (dim 0)'])

        # hacky...
        def patch_initial_params(self=fit):
            params = type(self).initial_params(self)
            a    = [val for key, val in params.items() if      'α' in key][0]
            logG = [val for key, val in params.items() if 'log(αΓ)' in key][0]
            params['α'] = a
            params['log(αΓ)'] = logG
            return params
        fit.initial_params = patch_initial_params

        for fitname in fit.fits_dict:
            fit.parameters[fitname+f' α (dim 0)'].fix_to = 'α'
            if fitname == 'minflux':
                fit.parameters[fitname+f' log(αΓ) (dim 0)'].fix_to = 'log(αΓ)'
            else: # not minflux, so correct for 2-loc
                def twoGref(params): return params['log(αΓ)']+np.log(2)
                fit.parameters[fitname+f' log(αΓ) (dim 0)'].fix_to = twoGref

        fits[treatment]['joints'][groupname] = fit

        bar.update()

bar.close()

0it [00:00, ?it/s]

  0%|          | 0/7352 [00:00<?, ?it/s]

  0%|          | 0/6199 [00:00<?, ?it/s]

  0%|          | 0/7096 [00:00<?, ?it/s]

  0%|          | 0/6534 [00:00<?, ?it/s]

In [7]:
fitres = {}
for treatment in ['ctrl', 'DRB', 'TSA', 'ICRF']:
    print()
    print(17*'=')
    print(f'|| {ct:>5s} {treatment:<5s} ||')
    print(17*'=')
    print()
    
    fitres[treatment] = {}
    for name in fits[treatment]['joints']:
        print(name)
        print('='*20)

        with nl.Parallelize():
            fitres[treatment][name] = fits[treatment]['joints'][name].run(show_progress=True)

        for key in fitres[treatment][name]['params']:
            print(key, fitres[treatment][name]['params'][key])
        print()


||  U2OS ctrl  ||

minflux


fit iterations: 0it [00:00, ?it/s]

minflux log(σ²) (dim 0) -8.568080618413742
α 0.3290621973683439
log(αΓ) -5.6140300321076015
minflux α (dim 0) 0.3290621973683439
minflux log(αΓ) (dim 0) -5.6140300321076015

SPT 100ms


fit iterations: 0it [00:00, ?it/s]

SPT-100ms log(σ²) (dim 0) -7.4198098863866555
α 0.35189445529692565
log(αΓ) -5.744069801513546
SPT-100ms α (dim 0) 0.35189445529692565
SPT-100ms log(αΓ) (dim 0) -5.0509226209536005

SPT 2s


...

SPT-2s log(σ²) (dim 0) -6.799328583888659
α 0.3219077459534748
log(αΓ) -6.219392558364268
SPT-2s α (dim 0) 0.3219077459534748
SPT-2s log(αΓ) (dim 0) -5.526245377804322

SPT


fit iterations: 0it [00:00, ?it/s]

SPT-100ms log(σ²) (dim 0) -7.5561351342433865
SPT-2s log(σ²) (dim 0) -8.346976067072045
α 0.286226237329936
log(αΓ) -6.121496914769338
SPT-100ms α (dim 0) 0.286226237329936
SPT-2s α (dim 0) 0.286226237329936
SPT-100ms log(αΓ) (dim 0) -5.428349734209393
SPT-2s log(αΓ) (dim 0) -5.428349734209393

minflux + SPT


fit iterations: 0it [00:00, ?it/s]

minflux log(σ²) (dim 0) -8.87807574687922
SPT-100ms log(σ²) (dim 0) -7.856396950532863
SPT-2s log(σ²) (dim 0) -17.493412855447602
α 0.25513873828257716
log(αΓ) -6.114631891813091
minflux α (dim 0) 0.25513873828257716
minflux log(αΓ) (dim 0) -6.114631891813091
SPT-100ms α (dim 0) 0.25513873828257716
SPT-2s α (dim 0) 0.25513873828257716
SPT-100ms log(αΓ) (dim 0) -5.421484711253146
SPT-2s log(αΓ) (dim 0) -5.421484711253146



In [8]:
nl.io.write.hdf5(fitres, f'../data/20250327_fitres_NPFit-aGparam_{ct}.h5')

## Profiler
Estimate credible intervals for point estimates from profile likelihood. __Attention: computationally expensive__

This can also move the point estimate, if we find better parameters while exploring

In [ ]:
fitres = nl.io.load.hdf5(f'../data/20250327_fitres_NPFit-aGparam_{ct}.h5')
mci = {}
for treatment in ['ctrl', 'DRB', 'TSA', 'ICRF']:
    print()
    print(17*'=')
    print(f'|| {ct:>5s} {treatment:<5s} ||')
    print(17*'=')
    print()
    
    mci[treatment] = {}
    for name in fits[treatment]['joints']:
        print(name)
        print('='*20)
        
        profiler = bayesmsd.Profiler(fits[treatment]['joints'][name], max_restarts=50)
        profiler.point_estimate = fitres[treatment][name]

        with nl.Parallelize():
            mci[treatment][name] = profiler.find_MCI(show_progress=True)

        for key in mci[treatment][name]:
            m, (cil, cih) = mci[treatment][name][key]
            print(f"{key:>25s} = {m:>6.3f} [{cil:>6.3f}, {cih:>6.3f}]")
        print()


||  U2OS ctrl  ||

minflux


profiler iterations: 0it [00:00, ?it/s]

  minflux log(σ²) (dim 0) = -8.568 [-8.594, -8.542]
                        α =  0.329 [ 0.324,  0.334]
                  log(αΓ) = -5.614 [-5.643, -5.585]

SPT 100ms


profiler iterations: 0it [00:00, ?it/s]

SPT-100ms log(σ²) (dim 0) = -7.420 [-7.479, -7.365]
                        α =  0.352 [ 0.340,  0.364]
                  log(αΓ) = -5.744 [-5.766, -5.722]

SPT 2s


...

profiler iterations: 0it [00:00, ?it/s]

SPT-100ms log(σ²) (dim 0) = -7.638 [-7.713, -7.569]
   SPT-2s log(σ²) (dim 0) = -19.230 [  -inf, -11.527]
                        α =  0.257 [ 0.252,  0.263]
                  log(αΓ) = -5.650 [-5.659, -5.641]

minflux + SPT


profiler iterations: 0it [00:00, ?it/s]

[bayesmsd.Profiler @ 1]  Warning: Found a better point estimate (2961726.6248392183 > 2961726.6230789674)
[bayesmsd.Profiler @ 1]  Will restart from there (50 remaining)


fit iterations: 0it [00:00, ?it/s]

In [ ]:
nl.io.write.hdf5(mci, f'../data/20250327_mci_NPFit-aGparam_{ct}.h5')